In [1]:
import requests
import pandas as pd
from datetime import datetime

In [2]:
res = requests.get('https://api.henleypassportindex.com/api/passports')
data = res.json()

In [3]:
code_list = [{'code': item.get('code'),'country':item.get('name')} for item in data if item.get('code') != '']
#sort by country name instead of code
code_list = sorted(code_list, key=lambda k: k['country'])

In [4]:
origin_lst = []
destination_lst = []
requirement = []
visa_free_count_lst = []
visa_required_count_lst = []
origin_for_count = []
visa_free = 'Visa Free'
visa_required = 'Visa Required'

for origin in code_list:
    origin_country = origin.get('country')
    origin_for_count.append(origin_country)
    count_vf = 0
    count_vr = 0
    res = requests.get('https://api.henleypassportindex.com/api/passports/' + origin.get('code') + '/countries')
    data = res.json()
    for destination in data['default']:
        destination_country = destination.get('name')
        origin_lst.append(origin_country)
        destination_lst.append(destination_country)
        is_visa_free = destination.get('pivot').get('is_visa_free')
        if str(is_visa_free) == "1":
            count_vf += 1
            requirement.append(visa_free)
        else:
            if str(origin_country) == str(destination_country):
                requirement.append("N/A")
            else:
                count_vr +=1
                requirement.append(visa_required)

    visa_free_count_lst.append(count_vf)
    visa_required_count_lst.append(count_vr)

In [5]:
today_date = datetime.today().strftime('%Y-%m-%d')
file_name = "henley-passport-index"+"-"+today_date+".csv"

pd_1 = pd.DataFrame({'Origin': origin_lst, 'Destination': destination_lst, 'Requirement': requirement})
pd_1.to_csv(file_name, index=False)

In [6]:
#filter unique countries in origin_lst
origin_lst = list(set(origin_lst))
#filter unique countries in destination_lst
destination_lst = list(set(destination_lst))

print(
    "Total number of countries in origin_lst: " + str(len(origin_lst)) ,
    "Total number of countries in destination_lst: " + str(len(destination_lst))
)

Total number of countries in origin_lst: 199 Total number of countries in destination_lst: 227


In [7]:
#filter countries not common in origin_lst and destination_lst
pd_1

,Origin,Destination,Requirement
0,Afghanistan,Afghanistan,N/A
1,Afghanistan,Albania,Visa Required
2,Afghanistan,Algeria,Visa Required
3,Afghanistan,American Samoa,Visa Required
4,Afghanistan,Andorra,Visa Required
...,...,...,...
45169,eSwatini,Vietnam,Visa Required
45170,eSwatini,Yemen,Visa Required
45171,eSwatini,Zambia,Visa Free
45172,eSwatini,Zimbabwe,Visa Free


In [8]:
today_date = datetime.today().strftime('%Y-%m-%d')
file_name = "henley-passport-index-count"+"-"+today_date+".csv"

pd_2 = pd.DataFrame({'Origin': origin_for_count, 'Visa Free': visa_free_count_lst, 'Visa Required': visa_required_count_lst})
pd_2.to_csv(file_name, index=False)

In [9]:
pd_2

,Origin,Visa Free,Visa Required
0,Afghanistan,27,199
1,Albania,117,109
2,Algeria,53,173
3,Andorra,170,56
4,Angola,50,176
...,...,...,...
194,Vietnam,56,170
195,Yemen,35,191
196,Zambia,70,156
197,Zimbabwe,66,160
